In [8]:
import pandas as pd
import sqlite3

In [11]:
connection = sqlite3.connect("cervezas.db")

crsr = connection.cursor()

In [10]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [13]:
# crear una tabla reparto

query1 = '''
DROP TABLE IF EXISTS reparto; 
'''
query2 = '''
CREATE TABLE IF NOT EXISTS reparto(
    CodE VARCHAR(2) NOT NULL,
    CodB VARCHAR(3) NOT NULL,
    CodC VarCHAR(2) NOT NULL,
    Fecha DATE NOT NULL,
    Cantidad SMALLINT,
    PRIMARY KEY(CodE, CodB, CodC, fecha)
)
'''
crsr.execute(query1)
crsr.execute(query2)

In [14]:
sql_query("SELECT * FROM sqlite_master WHERE type='table'")

,type,name,tbl_name,rootpage,sql
0,table,reparto,reparto,2,CREATE TABLE reparto(\n CodE VARCHAR(2) NOT...


In [15]:
query = '''
CREATE TABLE IF NOT EXISTS cervezas (
    CodC VARCHAR(2),
    Envase VARCHAR(32),
    Capacidad FLOAT(2),
    Stock INT(5),
    PRIMARY KEY (CodC)
)
'''

crsr.execute(query)

In [16]:
query = '''
CREATE TABLE IF NOT EXISTS bares (
    CodB VARCHAR(2),
    Nombre VARCHAR(32),
    Cif VARCHAR(32),
    Localidad VARCHAR(32),
    PRIMARY KEY (CodB)
)
'''

crsr.execute(query)

In [17]:
query = '''
CREATE TABLE IF NOT EXISTS empleados (
    CodE VARCHAR(2),
    Nombre VARCHAR(32),
    Sueldo INT(5),
    PRIMARY KEY (CodE)
)
'''

crsr.execute(query)

In [18]:
query = '''
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) 
VALUES  ('1','001','01','2005-10-21',240 ),
( '1','001','02','2005-10-21',48 ),
( '1','002','03','2005-10-22',60 ),
( '1','004','05','2005-10-22',4 ),
( '2','002','03','2005-10-22',48 ),
( '2','002','05','2005-10-23',2 ),
( '2','004','01','2005-10-23',480 ),
( '2','004','02','2005-10-24',72 ),
( '3','003','03','2005-10-24',48 ),
( '3','003','04','2005-10-25',20 )
;
'''
crsr.execute(query)

In [19]:
query = '''
INSERT OR REPLACE INTO bares VALUES ('001','Stop', '11111111X','Villa Botijo'),
('002', 'Las Vegas', '22222222Y', 'Villa Botijo'),
('003', 'Club Social', NULL, 'Las Ranas'),
('004','Otra Ronda','33333333Z','La Esponja')
'''
crsr.execute(query)

In [20]:
query = '''
INSERT OR IGNORE INTO Cervezas VALUES ('01','Botella',0.2,3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04','Botella',1,288),
('05','Barril',60,30)
'''
crsr.execute(query)

In [21]:
query = '''
INSERT OR REPLACE INTO empleados VALUES ('1','Prudencio Caminero',120000),
('2', 'Vicente Merario', 110000),
('3', 'Valentin Siempre', 100000)
'''
# CRUD Create Register(insert) Update Delete
crsr.execute(query)

In [29]:
query = '''
SELECT DISTINCT e.Nombre
FROM empleados e
LEFT JOIN reparto r
ON e.CodE = r.CodE
LEFT JOIN bares b
ON b.codB = r.CodB
WHERE r.fecha BETWEEN '17/10/2005' AND '23/10/2005' AND b.nombre = 'Stop'
'''

sql_query(query)

,Nombre,Fecha,bar
0,Prudencio Caminero,2005-10-21,Stop


In [30]:
#2
query = '''
SELECT DISTINCT bares.nombre, bares.cif
FROM bares
INNER JOIN reparto
ON bares.CodB= reparto.CodB
INNER JOIN cervezas
ON cervezas.CodC= reparto.CodC
WHERE cervezas.capacidad <1 AND cervezas.envase = 'Botella'
ORDER BY bares.Localidad
'''

sql_query(query)

,Nombre,Cif
0,Otra Ronda,33333333Z
1,Stop,11111111X


In [31]:
#3
query =('''
SELECT bares.nombre, cervezas.Envase,cervezas.Capacidad, reparto.Fecha,reparto.Cantidad
FROM reparto
INNER JOIN bares
ON reparto.CodB = bares.CodB
INNER JOIN cervezas
ON reparto.CodC = cervezas.CodC
INNER JOIN empleados
ON reparto.codE = empleados.CodE
WHERE empleados.nombre = 'Prudencio Caminero'
''')
sql_query(query)

,Nombre,Envase,Capacidad,Fecha,Cantidad
0,Stop,Botella,0.20,2005-10-21,240
1,Stop,Botella,0.33,2005-10-21,48
2,Las Vegas,Lata,0.33,2005-10-22,60
3,Otra Ronda,Barril,60.00,2005-10-22,4


In [32]:
#4
query = '''
SELECT DISTINCT b.nombre --, c.envase, c.capacidad

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
WHERE c.envase = 'Botella' AND c.capacidad IN (0.2, 0.33)

'''
sql_query(query)

,Nombre
0,Stop
1,Otra Ronda


In [33]:
#5
query = '''
SELECT e.nombre as nombreE, b.nombre as nombreB, c.envase

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
INNER JOIN empleadoS e
ON e.CodE = r.CodE
WHERE c.envase = 'Botella' AND b.nombre  IN ("Stop", "Las Vegas")

'''
sql_query(query)

,nombreE,nombreB,Envase
0,Prudencio Caminero,Stop,Botella
1,Prudencio Caminero,Stop,Botella


In [34]:
#5
query = '''
SELECT e.nombre as nombreE, b.nombre as nombreB, c.envase

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
INNER JOIN empleadoS e
ON e.CodE = r.CodE

GROUP BY e.nombre 
HAVING c.envase = 'Botella' AND b.nombre IN ('Stop', 'Las Vegas')
'''
sql_query(query)

,nombreE,nombreB,Envase
0,Prudencio Caminero,Stop,Botella


In [35]:
#6
query = '''
SELECT  e.nombre AS nombreE, b.nombre AS nombreB, b.localidad, count(distinct r.fecha)
FROM empleados e
INNER JOIN reparto r
ON e.CodE = r.CodE
INNER JOIN bares b
ON b.CodB = r.CodB
WHERE localidad != 'Villa Botijo'
GROUP BY e.nombre, b.nombre
'''
sql_query(query)

,nombreE,nombreB,Localidad,count(distinct r.fecha)
0,Prudencio Caminero,Otra Ronda,La Esponja,1
1,Valentin Siempre,Club Social,Las Ranas,2
2,Vicente Merario,Otra Ronda,La Esponja,2


In [36]:
#7
query = '''
SELECT t.nombre, t.localidad, max(t.maximo) AS maximo
FROM

(SELECT 
b.nombre, b.localidad, max(r.cantidad * c.capacidad) as maximo
FROM cervezas c
INNER JOIN reparto r
ON c.CodC = r.CodC
INNER JOIN bares b
ON b.CodB = r.CodB

GROUP BY b.nombre, b.localidad)  t


'''
sql_query(query)

,nombre,localidad,maximo
0,Otra Ronda,La Esponja,240.0
